In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
input_shape = (224, 224, 3)
num_classes = 4
batch_size = 32

In [27]:
# Set the paths to the training, test and validation data directories
train_dir = 'data/MSID_SP/train'
test_dir = 'data/MSID_SP/test'
val_dir = 'data/MSID_SP/validation'

In [28]:
# Create an instance of the ImageDataGenerator class for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)

# Create an instance of the ImageDataGenerator class for validation data (no augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create an instance of the ImageDataGenerator class for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)


Found 680 images belonging to 4 classes.
Found 245 images belonging to 4 classes.
Found 247 images belonging to 4 classes.


In [29]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a classification layer
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)

Epoch 1/10
22/22 [==============================] - 35s 1s/step - loss: 0.7989 - accuracy: 0.7265 - val_loss: 18.2187 - val_accuracy: 0.3469
Epoch 2/10
22/22 [==============================] - 32s 1s/step - loss: 0.4221 - accuracy: 0.8779 - val_loss: 9.5435 - val_accuracy: 0.3918
Epoch 3/10
22/22 [==============================] - 32s 1s/step - loss: 0.3800 - accuracy: 0.8824 - val_loss: 26.0420 - val_accuracy: 0.3429
Epoch 4/10
22/22 [==============================] - 32s 1s/step - loss: 0.2835 - accuracy: 0.9118 - val_loss: 12.1935 - val_accuracy: 0.4531
Epoch 5/10
22/22 [==============================] - 33s 1s/step - loss: 0.1647 - accuracy: 0.9485 - val_loss: 4.2820 - val_accuracy: 0.6163
Epoch 6/10
22/22 [==============================] - 33s 1s/step - loss: 0.2493 - accuracy: 0.9191 - val_loss: 15.8050 - val_accuracy: 0.4571
Epoch 7/10
22/22 [==============================] - 33s 1s/step - loss: 0.3630 - accuracy: 0.8735 - val_loss: 7.7440 - val_accuracy: 0.3673
Epoch 8/10
22/22

In [31]:
# evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

8/8 [==============================] - 2s 278ms/step - loss: 19.3719 - accuracy: 0.4332
Test loss: 19.371889114379883
Test accuracy: 0.43319839239120483
